In [2]:
using Pkg; for p in ("Embeddings","DataStructures","DataFrames","FileIO","LinearAlgebra","Knet","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using DataStructures,DataFrames,FileIO,Embeddings,LinearAlgebra
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data

In [3]:
#read data from saved file 
revs, W, W2, word_idx_map, vocab=load("TREC_sib.jld2","datas");
word_idx_map["ROOT"] = 0

0

In [14]:
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]
W2=Embed(300,length(vocab))

Embed(P(KnetArray{Float32,2}(10097,300)))

In [254]:
#how the source code is representing the question text as matrix 
function get_text_mat(t,word_idx_map,filter_h=5)
    #t the text of question
    x=[] # output matrix
    pad=filter_h -1 # padding number
    for i in collect(1:pad);push!(x,0);end #adding padding 
    words=split(t)
    #extract the unique id of words in the question text and adding it to the matrix 
    for w in words
        if w in keys(word_idx_map);push!(x,word_idx_map[w])
        else; @show w ;end
    end    

    while length(x)<56+2*pad    # accomplish 64 +1 size by adding zeros till finish 
            push!(x,0)
    end
    
    return  x
end

get_text_mat (generic function with 1 method)

In [15]:
#how the source code is representing the question tree as matrix 
function get_tree_rep(r,word_idx_map)
# question 
#@show t=r["tree"] #the tree of question
    each_sent=deepcopy(r)# output matrix
    for (j, each_word) in enumerate(each_sent[1:end-1])
        #@show (j, each_word)
            for (l, each_field) in enumerate(each_word)
           # @show (l, each_field)
                if each_field in keys( word_idx_map)
                #@show j,l ;
                    each_sent[j]=Array{Any,1}(each_sent[j])
                     each_sent[j][l] = word_idx_map[each_field]
                elseif each_field == 0
                    continue
                else
                    @show each_field
                end
            end
    end       
    return each_sent;
end

get_tree_rep (generic function with 1 method)

In [16]:
function train_dev_test(revs)
    s1,s2,s3=[],[],[]
    t1,t2,t3=[],[],[]
    for rev in revs
    sent =get_text_mat(rev["text"], word_idx_map,5)   
    push!(sent,rev["y"])
    sent_tensor = get_tree_rep(rev["tree"], word_idx_map)
        
    if rev["split"]==1
            push!(s1,KnetArray{Int}(sent))
            push!(t1,sent_tensor)
    elseif rev["split"]==2
            push!(s2,KnetArray{Int}(sent))
            push!(t2,sent_tensor)
    elseif rev["split"]==3
            push!(s3,KnetArray{Int}(sent))
            push!(t3,sent_tensor)
    end
end

    train = hcat([f1 for f1 in s1]...)
    test =hcat([f1 for f1 in s2]...)
    dev = hcat([f1 for f1 in s3]...)
    train_tensor = t1
    test_tensor = t2
    dev_tensor = t3
    return (train,test,dev),(train_tensor,test_tensor,dev_tensor)
end
dataset,datasetTensor=train_dev_test(revs);


UndefVarError: UndefVarError: get_text_mat not defined

dataset[1] --> train 65×5451 } text data
dataset[2]----> test 65×501  }
dataset[3] ----> dev 0x0     }

datasetTensor[1] ---> train_tensor 5451(46(10)))
datasetTensor[2] ---> test_tensor 501(46(10)))
datasetTensor[3] --->dev_tensor 0(0(0))